In [1]:
local_dir = '/home/neilor/SCIELO_DADOS/alacip/'
connect_cmd = "dbname='scielo_db' user='neilor' host='localhost' password='n1f2c3n1'"
%load_ext sql
%sql postgresql://neilor:n1f2c3n1@/scielo_db
import psycopg2 as pg
        

/home/neilor/anaconda3/lib/python3.6/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/home/neilor/anaconda3/lib/python3.6/site-packages/sql/magic.py:4: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Bool, Int, Unicode


### Atualiza Autores (nome, instituicao, sexo) em artigos_selecionados

In [2]:
def get_sql_update_autor_nome(num, nome, scielo_id):
    sql = "update alacip.artigos_selecionados " \
    +"set autor_"+num+"_nome = '"+nome+"'" \
    +" where scielo_id like '"+scielo_id+"';"
    return sql

def get_sql_update_autor_instituicao(num, instituicao, scielo_id):
    sql = "update alacip.artigos_selecionados " \
    +" set autor_"+num+"_vinculo_institucional = '"+instituicao+"'"  \
    +" where scielo_id like '"+scielo_id+"';"
    return sql

def get_sql_update_autor_sexo(num, sexo, scielo_id):
    sql = "update alacip.artigos_selecionados " \
    +" set autor_"+num+"_sexo = '"+sexo+"'"  \
    +" where scielo_id like '"+scielo_id+"';"
    return sql

def update_autores(id,conn):
    sql = "SELECT autor_nome_completo,autor_instituicao, autor_sexo \
    FROM alacip.autores_selecionados     \
    where artigo_scielo_id like '"+id+"';"

    autores = conn.cursor()
    autores.execute(sql)

    i=1
    for autor in autores:
        if i<11:
            nome = autor[0]
            instituicao = autor[1]
            sexo = autor[2]

            if nome != None:
                nome = nome.replace("'", "''")
                atu_nome = get_sql_update_autor_nome(str(i),nome, id)
                with conn:
                    with conn.cursor() as curs:
                        curs.execute(atu_nome)
            if instituicao != None:
                instituicao = instituicao.replace("'", "''")
                atu_instituicao = get_sql_update_autor_instituicao(str(i),instituicao, id)
                with conn:
                    with conn.cursor() as curs:
                        curs.execute(atu_instituicao)
            if sexo != None:
                atu_sexo = get_sql_update_autor_sexo(str(i),sexo, id)
                with conn:
                    with conn.cursor() as curs:
                        curs.execute(atu_sexo)  
                        
            i=i+1
        
    autores.close()

    
conn = pg.connect("dbname='scielo_db' user='neilor' host='localhost' password='n1f2c3n1'")
cursor = conn.cursor()
cursor.execute("select scielo_id from alacip.artigos_selecionados;")

for scielo_id in cursor:
    update_autores(scielo_id[0],conn)

cursor.close()
conn.close()   

print( " ATUALIZADOS AUTOR INSTITUICAO E SEXO")    


#print(get_lista_autores('S1415-47571998000100002'))

ProgrammingError: relation "alacip.autores_selecionados" does not exist
LINE 1: ...e_completo,autor_instituicao, autor_sexo     FROM alacip.aut...
                                                             ^


### ATUALIZA AUTORES BOLSISTAS EM artigos_selecionados

In [3]:
def get_sql_clear_bolsista(num):
    sql = "update alacip.artigos_selecionados  as a" \
    +" set autor_"+str(num)+"_pq = '0'" \
    +"     , bolsistas_pq = '0' " \
    +" ;"  
    return sql

def get_sql_update_bolsista(num):
    sql = "update alacip.artigos_selecionados  as a" \
    +" set autor_"+str(num)+"_pq = '1' " \
    +"     , bolsistas_pq = cast((cast(bolsistas_pq as int)+1) as varchar) " \
    +" from alacip.bolsistas as b" \
    +" where upper(a.autor_"+str(num)+"_nome) in (select upper(nome) from alacip.bolsistas)" \
    +" and  a.artigo_ano >= cast(extract('year' from b.inicio) as varchar)" \
    +" and  a.artigo_ano <= cast(extract('year' from b.termino) as varchar)" \
    +";"  
    return sql

for n in range(1,11):
    %sql {get_sql_clear_bolsista(n)}

for n in range(1,11):
    %sql {get_sql_update_bolsista(n)}

7200 rows affected.
7200 rows affected.
7200 rows affected.
7200 rows affected.
7200 rows affected.
7200 rows affected.
7200 rows affected.
7200 rows affected.
7200 rows affected.
7200 rows affected.
45 rows affected.
17 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.


In [ ]:
###  MARCA BOLSISTA EM ATORES SELECIONADOS

In [ ]:
def get_sql_clear_autor_pq():
    sql = "update alacip.atores_selecionados  as a" \
    +" set autor_pq = '0'" \
    +" ;"  
    return sql

def get_sql_update_autor_pq():
    sql = "update alacip.atores_selecionados  as a" \
    +" set autor_pq = '1' " \
    +" from alacip.bolsistas as b" \
    +" ,alacip.artigos_selecionados as ar"\
    +" where upper(a.autor_nome_completo) in (select upper(nome) from alacip.bolsistas)" \
    +" and  ar.artigo_ano >= cast(extract('year' from b.inicio) as varchar)" \
    +" and  ar.artigo_ano <= cast(extract('year' from b.termino) as varchar)" \
    +";"  
    return sql

sql_eh_bolsista = """
select autor_nome_completo, count(*) as qtd from alacip.autores_selecionados
where autor_nome_completo in
(select nome from alacip.bolsistas) 
group by autor_nome_completo

"""

%sql {get_sql_clear_autor_pq()}

%sql {get_sql_update_autor_pq()}